In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch

from noteify.core.config import SAMPLE_RATE
from noteify.core.datasets import (MaestroDataset, MaestroDatasetProcessed, MusicAugmentor,
                                   MusicSegmentSampler, get_music_dataloader)
from noteify.core.training import make_optimizer, make_scheduler, train_model
from noteify.core.models import TranscriptionNN
from noteify.core.utils import plot_audio, plot_roll_info
from noteify.utils import get_rel_pkg_path

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("Using the GPU!")
else:
    print("Warning: Could not find GPU! Using CPU only")

Using the GPU!


In [3]:
data_dir = get_rel_pkg_path("maestro/maestro-v3.0.0")
save_dir = get_rel_pkg_path("weights/")

In [4]:
raw_dataset_train = MaestroDataset(data_dir, train=True, reduce_record_factor=1.5)
raw_dataset_test = MaestroDataset(data_dir, train=False, reduce_record_factor=1.5)

 13%|██████████▎                                                                     | 83/641 [00:00<00:00, 826.97it/s]

Loading audio


100%|███████████████████████████████████████████████████████████████████████████████| 118/118 [00:00<00:00, 830.84it/s]

Loading audio


In [5]:
batch_size = 32

In [6]:
dataset_train = MaestroDatasetProcessed(raw_dataset_train, augmentor=MusicAugmentor())
sampler_train = MusicSegmentSampler(dataset_train, batch_size, num_batches=None, random_start_times=True)
dataloader_train = get_music_dataloader(dataset_train, sampler_train, num_workers=8, pin_memory=True)

dataset_test = MaestroDatasetProcessed(raw_dataset_test)
sampler_test = MusicSegmentSampler(dataset_test, batch_size)
dataloader_test = get_music_dataloader(dataset_test, sampler_test, num_workers=None, pin_memory=True)

dataloaders = {'train': dataloader_train, 'test': dataloader_test}

In [7]:
model = TranscriptionNN()
model = model.to(device)
past_train_weights = os.path.abspath(os.path.join(save_dir, "Experiment 12-08-2020 05-15AM/Weights Best.pckl"))
model.load_state_dict(torch.load(past_train_weights))

CQT kernels created, time used = 0.1444 seconds


<All keys matched successfully>

In [8]:
optimizer = make_optimizer(model, lr=0.001)

In [9]:
num_epochs = 100
scheduler = make_scheduler(optimizer, [60], gamma=0.5)

In [ ]:
tracker = train_model(device=device,
                      model=model,
                      dataloaders=dataloaders,
                      optimizer=optimizer,
                      lr_scheduler=scheduler,
                      save_model=True,
                      save_dir=save_dir,
                      save_best=True,
                      save_all=False,
                      num_epochs=num_epochs)

  0%|                                                                                         | 0/1809 [00:00<?, ?it/s]

Epoch 0/99
----------


Avg. Loss: 0.0972, Total Loss: 0.0807, Loss Parts: [0.0561, 0.0100, 0.0146]: 100%|█| 1809/1809 [32:10<00:00,  1.07s/it]


Training Loss: 0.0972
Test statistics: {'frame_avg_precision': 0.7614009502072071, 'reg_onset_mae': 0.12908941507339478, 'reg_offset_mae': 0.1244644820690155}


  0%|                                                                                         | 0/1809 [00:00<?, ?it/s]

...

Avg. Loss: 0.0929, Total Loss: 0.0909, Loss Parts: [0.0687, 0.0090, 0.0131]: 100%|█| 1809/1809 [32:20<00:00,  1.07s/it]


Training Loss: 0.0929
Test statistics: {'frame_avg_precision': 0.7566443224551089, 'reg_onset_mae': 0.11588550359010696, 'reg_offset_mae': 0.12767168879508972}


  0%|                                                                                         | 0/1809 [00:00<?, ?it/s]


Epoch 2/99
----------


Avg. Loss: 0.0908, Total Loss: 0.0911, Loss Parts: [0.0691, 0.0094, 0.0126]:  22%|▍ | 389/1809 [07:22<19:49,  1.19it/s]

In [ ]:
%qtconsole